Created by Beyza Balota, 31232
(04.04.2024)

# Importing the necessary libraries

In [1]:
import pandas as pd

# Loading the data

In [2]:
# Load the multiple choice responses with the correct encoding and used low_memory option to avoid DtypeWarning cause I kept getting that error
multiple_choice_responses_df = pd.read_csv('multipleChoiceResponses.csv', encoding='ISO-8859-1', low_memory=False)
conversion_rates_df = pd.read_csv('conversionRates.csv', encoding='ISO-8859-1')

# Questions Part

# 2.a 

What is the probability that a respondent is currently employed as a Programmer given they use C/C++ at work?

In [3]:
# Respondents who use C/C++ at work. Do not considering 'never'
cpp_users = multiple_choice_responses_df[multiple_choice_responses_df['WorkToolsFrequencyC'].notnull() & (multiple_choice_responses_df['WorkToolsFrequencyC'] != 'Never')]

# Respondents who are Programmers and use C/C++.
cpp_programmers = cpp_users[cpp_users['CurrentJobTitleSelect'] == 'Programmer']

# Probability calculation
p_cpp_programmer = len(cpp_programmers) / len(cpp_users) if len(cpp_users) > 0 else 0

percentage = "{:.2%}".format(p_cpp_programmer)
print(percentage)

2.27%


# 2.b
What is the probability that a respondent is a Data Scientist given they have majored in computer science, mathematics or statistics?


In [4]:
interest = ['Computer Science', 'Mathematics or statistics', 'Mathematics', 'Statistics']

# Respondents who matches the interests
interest_filtered = multiple_choice_responses_df[multiple_choice_responses_df['MajorSelect'].isin(interest)]

# Respondents within the interests and also Data Scientists
data_scientists = interest_filtered[interest_filtered['CurrentJobTitleSelect'] == 'Data Scientist']

probability = len(data_scientists) / len(interest_filtered) if len(interest_filtered) > 0 else 0

percentage = "{:.2%}".format(probability)
print(percentage)

15.97%


# 2.c 
What is the probability that a respondent works in the Technology industry given that they earn more than 40,000 USD annually?


In [5]:
# Converting salaries to USD
merged_df = multiple_choice_responses_df.merge(conversion_rates_df, left_on='CompensationCurrency', right_on='originCountry', how='left')

# Making sure that the compensation amount is numeric
merged_df['CompensationAmount'] = pd.to_numeric(merged_df['CompensationAmount'], errors='coerce')

# Respondents earning $40,000+ and working in the technology industry
merged_filtered = merged_df[(merged_df['CompensationAmount'] * merged_df['exchangeRate'] > 40000) & (merged_df['EmployerIndustry'].notnull())]
technology_workers = merged_filtered[merged_filtered['EmployerIndustry'] == 'Technology']


probability = len(technology_workers) / len(merged_filtered) if len(merged_filtered) > 0 else 0
percentage = "{:.2%}".format(probability)
print(percentage)

19.64%


# 2.d 
What is the joint probability of a respondent being over 30 years old and having a at least a Bachelor’s degree?

In [6]:
degrees = ['Bachelor’s degree', 'Master’s degree', 'Doctoral degree', 'Professional degree']

# Respondents who are 30+ and have at least bachelors degree
merged_criteria = multiple_choice_responses_df[(multiple_choice_responses_df['Age'] > 30) & (multiple_choice_responses_df['FormalEducation'].isin(degrees))]

probability = len(merged_criteria) / len(multiple_choice_responses_df)
percentage = "{:.2%}".format(probability)
print(percentage)

12.81%


# 2.d 
What is the probability that a respondent is a Data Scientist who majored in Computer Science, Mathematics or statistics?

In [7]:
# Data Scientists respondents
data_scientists = multiple_choice_responses_df[multiple_choice_responses_df['CurrentJobTitleSelect'] == 'Data Scientist']

# Respondents studying computer science, mathematics, statistics
interest = ['Computer Science', 'Mathematics or statistics', 'Mathematics', 'Statistics']
data_scientist_with_major = data_scientists[data_scientists['MajorSelect'].isin(interest)]


probability = len(data_scientist_with_major) / len(multiple_choice_responses_df) if len(multiple_choice_responses_df) > 0 else 0
percentage = "{:.2%}".format(probability)
print(percentage)

6.32%


# 2.f 
What is the joint probability that a respondent is from France, earns less than 100,000 USD annually, and uses Cross-Validation Often or Most of the time?

In [8]:
# Exchange rates for salary conversion
merged_df = multiple_choice_responses_df.merge(conversion_rates_df, left_on='CompensationCurrency', right_on='originCountry', how='left')

# Making sure that the compensation amount is numeric
merged_df['CompensationAmount'] = pd.to_numeric(merged_df['CompensationAmount'], errors='coerce')

# Respondents from France
france_respondents = merged_df[merged_df['Country'] == 'France']

# Respondents earning <100,000k
earnings = france_respondents[(france_respondents['CompensationAmount'] * france_respondents['exchangeRate'] < 100000)]

# Respondents using cross validation
cross_validation_users = earnings[earnings['WorkMethodsFrequencyCross-Validation'].isin(['Often', 'Most of the time'])]

joint_probability = len(cross_validation_users) / len(merged_df) if len(merged_df) > 0 else 0
percentage = "{:.2%}".format(joint_probability)
print(percentage)


0.36%


# 2.g 
What is the probability that a respondent uses C/C++ at work given that they are employed as a Programmer? (Hint: Use your findings from Question 2a).

In [9]:
# Respondents using c/c++
cpp_users = multiple_choice_responses_df[multiple_choice_responses_df['WorkToolsFrequencyC'].notnull() & (multiple_choice_responses_df['WorkToolsFrequencyC'] != 'Never')]

# P(C/C++)
p_cpp = len(cpp_users) / len(multiple_choice_responses_df) if len(multiple_choice_responses_df) > 0 else 0

# P(Programmer)
programmers = multiple_choice_responses_df[multiple_choice_responses_df['CurrentJobTitleSelect'] == 'Programmer']
p_programmer = len(programmers) / len(multiple_choice_responses_df) if len(multiple_choice_responses_df) > 0 else 0

# P(Programmer | C/C++) from the question 2a
p_programmer_given_cpp = 0.022651565622918056

# P(C/C++ | Programmer) using Bayes Theorem
p_cpp_given_programmer = (p_programmer_given_cpp * p_cpp) / p_programmer if p_programmer > 0 else 0
percentage = "{:.2%}".format(p_cpp_given_programmer)
print(percentage)

7.36%


# 2.h 
Given the probability of a respondent wearing glasses is 0.15, and the probability of a respondent wearing glasses given they have a PhD is 0.25, find the probability of a respondent having a PhD given that they wear glasses.

In [10]:
# PHD respondents
phd_respondents= multiple_choice_responses_df[multiple_choice_responses_df['FormalEducation'].str.contains('Doctoral degree', na=False)]
total_respondents = multiple_choice_responses_df[multiple_choice_responses_df['FormalEducation'].notnull()]

# P(PhD) without NA values
p_phd= len(phd_respondents) / len(total_respondents)

glasses = 0.15
glasses_given_phd = 0.25

# P(PhD | Glasses)
phd_given_glasses = (glasses_given_phd * p_phd) / glasses
percentage = "{:.2%}".format(phd_given_glasses)
print(percentage)

26.05%
